In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from NN import NLayerNetwork

In [2]:
df = pd.read_excel('titanic.xls')
columns_to_drop = ['name','ticket', 'home.dest']
df = df.drop(columns_to_drop, axis=1)

df['embarked'] = pd.factorize(df['embarked'])[0]

df['cabin'] = df['cabin'].fillna(df['cabin'].mode())
df['cabin'] = pd.factorize(df['cabin'])[0]

df['sex'].value_counts(dropna=False)
df['sex'] = df['sex'].map(lambda x: 1 if x == 'male' else 0)
df['sex'].value_counts()

df['fare'] = df['fare'].fillna(df['fare'].mean())

df['age'] = df['age'].fillna(df['age'].mean())

df['boat'] = df['boat'].fillna(0)
df['boat'] = pd.factorize(df['boat'])[0]

df['body'] = df['body'].fillna(df['body'].mean())

df.head()

,pclass,survived,sex,age,sibsp,parch,fare,cabin,embarked,boat,body
0,1,1,0,29.0000,0,0,211.3375,0,0,0,160.809917
1,1,1,1,0.9167,1,2,151.5500,1,0,1,160.809917
2,1,0,0,2.0000,1,2,151.5500,1,0,2,160.809917
3,1,0,1,30.0000,1,2,151.5500,1,0,2,135.000000
4,1,0,0,25.0000,1,2,151.5500,1,0,2,160.809917


In [3]:
Y = np.array(df.iloc[:, 1:2])
print(Y.shape)
X = np.array(df.iloc[:, 2:])
print(X.shape)

(1309, 1)
(1309, 9)


In [4]:
model = NLayerNetwork(X, Y, [9, 5, 3, 1], learning_rate=0.001, iterations=200000)

In [5]:
model.fit()

Cost after iteration 0: 0.6931462658352523
Cost after iteration 10000: 0.2293737176884488
Cost after iteration 20000: 0.1846795224924257
Cost after iteration 30000: 0.28396438110851224
Cost after iteration 40000: 0.20079450045890698
Cost after iteration 50000: 0.17044892976723747
Cost after iteration 60000: 0.16490053541617453
Cost after iteration 70000: 0.16311929661314703
Cost after iteration 80000: 0.16565531170591852
Cost after iteration 90000: 0.18262255800944344
Cost after iteration 100000: 0.27911307469644486
Cost after iteration 110000: 0.1805024566724443
Cost after iteration 120000: 0.1616224508071373
Cost after iteration 130000: 0.20724330897195692
